In [106]:
import pandas as pd
import seaborn
from IPython import embed
from matplotlib import pyplot as plt
from scipy import optimize
import numpy as np


In [120]:
matches_df = pd.read_csv("../../Data/dartsdatabase/Averages.csv")

In [121]:
matches_df.head()

,Unnamed: 0,player_name,date,event,category,event_round,result,opponent,score,average
0,0,Michael van Gerwen,19/08/2018,Brisbane Masters,PDC U,Final,Lost,Rob Cross,6 V 11,98.41
1,1,Michael van Gerwen,19/08/2018,Brisbane Masters,PDC U,Semi Final,Won,Raymond van Barneveld,11 V 9,96.81
2,2,Michael van Gerwen,18/08/2018,Brisbane Masters,PDC U,Quarter Final,Won,Kyle Anderson,10 V 3,98.44
3,3,Michael van Gerwen,17/08/2018,Brisbane Masters,PDC U,Last 16,Won,Mark Cleaver,6 V 2,99.15
4,4,Michael van Gerwen,12/08/2018,Melbourne Masters,PDC U,Semi Final,Lost,Peter Wright,7 V 11,103.47


In [122]:
def outcome_map(result):
    if result == "Won":
        return 1.0
    elif result == "Lost":
        return 0.0
    else:
        return 0.5

#Drop matches that haven't yet happened
matches_df = matches_df[matches_df['result'] != 'fixture']

#Why are there some dups? Need to investigate
matches_df.drop_duplicates(inplace=True)

#Map outcomes to numerical values
matches_df['Outcome'] = matches_df['result'].apply(outcome_map)

In [123]:
#Rename columns - same as other versions
matches_df.rename(index=str, columns={"player_name": "Player 1", "opponent": "Player 2","date":"Date"},inplace=True)

matches_df['Tourn'] = matches_df['event'].str.upper()

matches_df['score'] =matches_df['score'].str.replace(' V ',':')



In [124]:
matches_df['Tourn'] = matches_df['Tourn'].str.replace('GRAND SLAM OF DARTS','GRAND SLAM')
#Only keep relevant columns
elo_df = matches_df.loc[:,['Player 1','Player 2','Outcome','Date','Tourn','score','averege']]


/Users/oli/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [125]:
#Drop mirror image fixtures. Quite crude, need to check
already = []
for player_name in elo_df['Player 1'].unique():
    #Keep if - player name not current one, or rival not in 'already' group. 
    elo_df = elo_df[(elo_df['Player 1'] != player_name) | (-elo_df['Player 2'].isin(already))]
    already.append(player_name)



#Keep surname only
elo_df['Surname 1'] = elo_df['Player 1'].str.split(' ',n=1).str.get(1).str.upper().str.strip()
elo_df['Surname 2'] = elo_df['Player 2'].str.split(' ',n=1).str.get(1).str.upper().str.strip()


In [126]:
#Turn date into datetime variable
elo_df['Date'] = pd.to_datetime(elo_df['Date'])

elo_df['year'] =elo_df['Date'].dt.year


In [127]:
#Merge on odds data - quite crude.

odds_df = pd.read_csv("../../Data/oddschecker/Darts_odds.csv")

def convert_frac_to_prob(frac):
    if "/" in frac:
        num, denom = frac.split("/")
        return int(denom) / (int(num) + int(denom))

odds_df['p1_prob'] = odds_df['odds_1'].apply(convert_frac_to_prob)
odds_df['p2_prob'] = odds_df['odds_2'].apply(convert_frac_to_prob)


date_cleaning = [
    ' - Play Offs',
    ' - Second stage',
    ' - First stage',
    ' - Winners stage',
    ' - Preliminary'
]

for str_x in date_cleaning:
        odds_df['date'] = odds_df['date'].str.replace(str_x,'')
    
        
odds_df['Date'] = pd.to_datetime(odds_df['date'])


In [128]:




odds_df['Player 1'] = odds_df['players'].str.split(' - ',n=1).str.get(0)
odds_df['Player 2'] = odds_df['players'].str.split(' - ',n=1).str.get(1)

odds_df['Surname 1'] = odds_df['Player 1'].str.slice(0,-2).str.upper().str.strip()
odds_df['Surname 2'] = odds_df['Player 2'].str.slice(0,-2).str.upper().str.strip()

#clean player names
surname_cleaning = [
    ("O'","O " ),
    ("RODRIGUEZ ROW.","RODRIGUEZ"),
    ("DOBEY C","DOBEY")
]

for pair_x in surname_cleaning:
    odds_df['Surname 1'] = odds_df['Surname 1'].str.replace(pair_x[0], pair_x[1])
    odds_df['Surname 2'] = odds_df['Surname 2'].str.replace(pair_x[0], pair_x[1])

odds_df['Tourn']=odds_df['tournament_name'].str.split('/').str.get(-1).str.replace("-"," ").str.upper()



In [129]:
#Same odds, the opposite way around
opp_df = odds_df.copy()

opp_df.rename(index=str, \
columns= {"odds_1": "odds_2", "odds_2": "odds_1","Player 1":"Player 2","Player 2":"Player 1","Surname 1":"Surname 2", \
"Surname 2":"Surname 1"},inplace=True)

#Doesn't work for games > 10.
opp_df['score']=opp_df['score'].apply(lambda x: x[::-1])

odds_df = odds_df.append(opp_df,ignore_index=True,sort=False)
        


In [130]:
merged_df = pd.merge(odds_df, elo_df,  how='right',left_on=['Surname 1','Surname 2','Date'],right_on=['Surname 1','Surname 2','Date'],indicator=True)

In [131]:
merged_df['_merge'].value_counts()

right_only    11203
both           3615
left_only         0
Name: _merge, dtype: int64

In [93]:
def count_non_matches(df1,df2,col):
    
    first = df1[-df1[col].isin(df2[col])][col].value_counts()
    second = df2[col].value_counts()
    print('first - non matches')
    print(first)
    print()
    print('second - all')
    print(second)
    
#count_non_matches(odds_df,elo_df,'Surname 1')

elo_df[elo_df['Surname 1'].str.match('STOMPE')]['Surname 1']

Series([], Name: Surname 1, dtype: object)

Index(['odds_1', 'odds_2', 'players', 'score', 'tournament_name', 'year',
       'p1_prob', 'p2_prob', 'Player 1', 'Player 2', 'Surname 1', 'Surname 2',
       'Tourn'],
      dtype='object')